<a href="https://colab.research.google.com/github/orbenishay/pandas-project/blob/master/Bicycle_6_Hagai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import pandas as pd
import matplotlib.pyplot as plt # visualization, EDA
import numpy as np  # Mathematical calculations
%matplotlib inline


In [5]:
if 'google.colab' in sys.modules:
    from google.colab import files
    uploaded = files.upload()

Saving weather.csv to weather.csv


In [42]:
# Hagai's Analysis:
# -----------------------------------------------------------------------------------------
# Read csv's
df_trip = pd.read_csv("trip.csv",
                   index_col='id', parse_dates=True)
df_trip['start_date'] = pd.to_datetime(df_trip['start_date'], errors='coerce', dayfirst=True)
df_trip['end_date'] = pd.to_datetime(df_trip['end_date'], errors='coerce', dayfirst=True)
df_trip['date_day']=df_trip['start_date'].dt.date
df_trip['duration']=df_trip['end_date']-df_trip['start_date']
# df_trip.head
df_station = pd.read_csv("station.csv", index_col = 'id')
# df_station.head()
df_bikes = pd.read_csv("bikes.csv", index_col = 'bike_id')
# df_bikes.head()
df_weather = pd.read_csv("weather.csv", parse_dates=['date'])
df_weather['date'] = pd.to_datetime(df_weather['date'], errors='coerce')
df_weather['date']=df_weather['date'].dt.date
# df_weather.head()

# -----------------------------------------------------------------------------------------
# Merges
df_trip_weather = df_trip.merge(df_weather, how='left', left_on = ['zip_code', 'date_day'], right_on=['zip_code','date'])
# df_trip_weather.head()
df_trip_weather_station = df_trip_weather.merge(df_station, how='left', left_on = 'start_station_id', right_on='id')
# df_trip_weather_station.head()
df_all = df_trip_weather_station.merge(df_bikes, how='left', left_on = 'bike_id', right_on='bike_id')
df_all.head()

# ----------------------------------------------------------------------------------------
# Year/Month aggregation
df_all['year'] = pd.DatetimeIndex(df_all['start_date']).year
df_all['month'] = pd.DatetimeIndex(df_all['start_date']).month
df_all['hour'] = pd.DatetimeIndex(df_all['start_date']).hour
df_all['weekday'] = pd.DatetimeIndex(df_all['start_date']).weekday
df_group=df_all.groupby(['month','year']).agg({'start_date':'count'}).unstack(0)
df_group.plot(kind='bar', stacked=False, figsize=(12, 6), by='year')

# ----------------------------------------------------------------------------------------
# Duration
df_all.boxplot(column='duration', by='city')
df_all['duration'].describe()
# df_all.groupby(['city']).agg({'duration_minutes':'mean'}).head()

# ------------------------------------------------------------------------------------------
# rents distribution by city

df_group_city_station=df_all.groupby(['city','station_name']).agg({'start_date':'count'})
df_group_city_station.boxplot(column='start_date', by='city')
df_group_city_station.groupby('city').describe().unstack(0)

# ------------------------------------------------------------------------------------------
# count rent by hour & weekday

# ------------------------------------------------------------------------------------------
# Popular Bike Type brand & color
df_group_bike_trips=df_all.groupby(['bike_type','bike_brand']).agg({'start_date':'count'})
df_group_bike_trips.head()
df_group_bike_trips.unstack(-1).plot(kind='bar', figsize=(12, 6), subplots=False)


df_all.describe()




,start_station_id,end_station_id,bike_id,zip_code,duration,mean_temperature_f,mean_humidity,mean_visibility_miles,mean_wind_speed_mph,cloud_cover,wind_dir_degrees,lat,long,dock_count
count,95299.000000,95299.000000,95299.000000,95299.000000,95299,95288.000000,95086.000000,95214.000000,95299.000000,95299.000000,95299.000000,95299.000000,95299.000000,95299.000000
mean,57.627719,57.999328,422.835465,94160.650930,0 days 00:41:34.631633,60.795021,68.331647,9.592949,7.549313,3.570992,259.180107,37.735259,-122.352862,28.114314
std,17.364316,17.637250,158.655081,223.719255,0 days 01:21:20.885424,6.203300,10.166949,1.159041,3.598821,2.310179,75.436105,0.132828,0.131644,9.583180
min,2.000000,2.000000,9.000000,94041.000000,0 days 00:01:00,38.000000,24.000000,4.000000,0.000000,0.000000,0.000000,37.329732,-122.418954,5.000000
25%,51.000000,51.000000,327.000000,94107.000000,0 days 00:23:00,56.000000,63.000000,10.000000,5.000000,1.000000,254.000000,37.776600,-122.402717,18.000000
50%,63.000000,64.000000,436.000000,94107.000000,0 days 00:39:00,60.000000,69.000000,10.000000,8.000000,4.000000,278.000000,37.781752,-122.395470,30.000000
75%,69.000000,69.000000,546.000000,94107.000000,0 days 00:54:00,66.000000,74.000000,10.000000,10.000000,5.000000,301.000000,37.788446,-122.392738,35.000000
max,84.000000,84.000000,878.000000,95113.000000,8 days 09:20:00,84.000000,96.000000,20.000000,23.000000,8.000000,2772.000000,37.804770,-121.877349,43.000000
